In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# 1. Import data

In [38]:
df_all=pd.read_csv('s3://niu00056-msba6330/Final Project/BookRatings.csv',sep=';')
df=df_all.iloc[:9000,:]

# 2. Tidy data into book rating format

In [39]:
df=df.pivot(index='ISBN',columns='User-ID',values='Rating')
df[pd.isnull(df)]=0

In [40]:
df.columns=['user_'+str(i) for i in df.columns]
df.index=['book_'+i for i in df.index]

In [41]:
df1=df.copy()

# 3. Recommender System

In [42]:
#Recommended Books
def recommend_books(user, num_recommended_books):
  print('The list of the Books {} Has Watched \n'.format(user))
  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  print('\n')
  recommended_books = []
  for m in df[df[user] == 0].index.tolist():
    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_books.append((m, predicted_rating))
  sorted_rm = sorted(recommended_books, key=lambda x:x[1], reverse=True)
  print('The list of the Recommended Books \n')
  rank = 1
  for recommended_book in sorted_rm[:num_recommended_books]:
    print('{}: {} - predicted rating:{}'.format(rank, recommended_book[0], recommended_book[1]))
    rank = rank + 1

In [43]:
#Recommeder System
# store the original dataset in 'df', and create the copy of df, df1 = df.copy().
def book_recommender(user, num_neighbors, num_recommendation):
  number_neighbors = num_neighbors

  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(df.values)
  distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

  user_index = df.columns.tolist().index(user)

  for m,t in list(enumerate(df.index)):
    if df.iloc[m, user_index] == 0:
      sim_books = indices[m].tolist()
      book_distances = distances[m].tolist()
    
      if m in sim_books:
        id_book = sim_books.index(m)
        sim_books.remove(m)
        book_distances.pop(id_book) 

      else:
        sim_books = sim_books[:num_neighbors-1]
        book_distances = book_distances[:num_neighbors-1]
           
      book_similarity = [1-x for x in book_distances]
      book_similarity_copy = book_similarity.copy()
      nominator = 0

      for s in range(0, len(book_similarity)):
        if df.iloc[sim_books[s], user_index] == 0:
          if len(book_similarity_copy) == (number_neighbors - 1):
            book_similarity_copy.pop(s)
          
          else:
            book_similarity_copy.pop(s-(len(book_similarity)-len(book_similarity_copy)))
            
        else:
          nominator = nominator + book_similarity[s]*df.iloc[sim_books[s],user_index]
          
      if len(book_similarity_copy) > 0:
        if sum(book_similarity_copy) > 0:
          predicted_r = nominator/sum(book_similarity_copy)
        
        else:
          predicted_r = 0

      else:
        predicted_r = 0
        
      df1.iloc[m,user_index] = predicted_r
  recommend_books(user, num_recommendation)

In [45]:
book_recommender('user_276729',3,3)

The list of the Books user_276729 Has Watched 

book_052165615X
book_0521795028


The list of the Recommended Books 

1: book_0002231115 - predicted rating:0.0
2: book_0002232766 - predicted rating:0.0
3: book_0002240114 - predicted rating:0.0
